#Fast Food SVM
Application of the Fast Food kernel expansion algorithm to SVMs.

In [80]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
#import pylab as pl
#import theano
#import theano.tensor as T
import numpy as np
from numpy.linalg import norm
#import pandas as pd
#import sklearn.datasets
from sklearn import svm
from scipy.linalg import hadamard
from scipy.special import gammaincinv
from functools import partial
from collections import namedtuple

VERBOSE = True  # increases verbosity of some outputs

d = 100 # dimension of input pattern
n = d*20 # basis number used for approximation
sgm = 10 # bandwidth for Gaussian kernel

# random number generator
rng = np.random.RandomState()

# load data
X1 = rng.normal(size=(d,2))
X2 = rng.normal(size=(d,3))
print('Shape of X1:',X1.shape,'; shape of X2:',X2.shape)

# pad with zeros so d is nearest power of 2
# N = X.shape[0]
# l = X.shape[1]
# print('Original shape of X: n =',N,', d =',l)
# d = int(2 ** np.ceil(np.log2(l)))
# if d != l:  # pad only if needed
#     print('Padding input from d =',l,'to d =',d)
#     X = np.pad(X,((0,d-l),(0,0)),mode='constant',constant_values=0)
    
# convert to shared tensors
#X = theano.shared(X,name='X',borrow=True)
#y = theano.shared(y,name='y',borrow=True)

Shape of X1: (100, 2) ; shape of X2: (100, 3)


In [81]:
FFPara = namedtuple('FFPara', 'B G PI S')
def fastfood_params(n,d):
    d0 = d
    l = int(np.ceil(np.log2(d)))
    d = 2**l
    k = int(np.ceil(n/d))
    n = d*k
#    print('d0 =',d0,', d =',d)
    
    B = []
    G = []
    PI = []
    S = []
    for ii in xrange(k):
        B_ii  = rng.choice([-1,1],size=d)
        G_ii  = rng.normal(size=d)
        PI_ii = rng.permutation(d)
        
        B.append(B_ii)
        G.append(G_ii)
        PI.append(PI_ii)
        
        p1 = rng.uniform(size=d)
        p2 = d/2
#        print('p1 =',p1,'; p2 =',p2)
        T = gammaincinv(p2,p1)
#        print('T1 =',T)
        T = (T*2) ** (1/2)
#        print('T2 =',T)
        s_i = T * norm(G,'fro')**(-1)
#        print('s_i =', s_i)
        S_ii = s_i
        S.append(S_ii)
    
    S1 = np.zeros(n)
    for ii in xrange(k):
        S1[ii*d:(ii+1)*d] = S[ii]
    
#    print('Shape of B:',len(B),', B[0]:',B[0].shape)
    
    return FFPara(B, G, PI, S1)
print('Ready to generate fastfood params')

Ready to generate fastfood params


In [82]:
def bit_reverse_traverse(a):
    # (c) 2014 Ryan Compton
    # ryancompton.net/2014/06/05/bit-reversal-permutation-in-python/
    n = a.shape[0]
    assert(not n&(n-1) ) # assert that n is a power of 2
    if n == 1:
        yield a[0]
    else:
        even_index = np.arange(int(n/2))*2
        odd_index = np.arange(int(n/2))*2 + 1
        for even in bit_reverse_traverse(a[even_index]):
            yield even
        for odd in bit_reverse_traverse(a[odd_index]):
            yield odd

def get_bit_reversed_list(l):
    # (c) 2014 Ryan Compton
    # ryancompton.net/2014/06/05/bit-reversal-permutation-in-python/
    n = len(l)
#    print('n=',n)
    indexs = np.arange(n)
    b = []
    for i in bit_reverse_traverse(indexs):
        b.append(l[i])
    return b

def FWHT(X):
    # Fast Walsh-Hadamard Transform for 1D signals
    # of length n=2^M only (non error-proof for now)
    x=get_bit_reversed_list(X)
    x=np.array(x)
    N=len(X)
 
    for i in range(0,N,2):
        x[i]=x[i]+x[i+1]
        x[i+1]=x[i]-2*x[i+1]
 
    L=1
    y=np.zeros_like(x)
    for n in range(2,int(np.log2(N))+1):
        M=2**L
        J=0; K=0
        while(K<N):
            for j in range(J,J+M,2):
                y[K]   = x[j]   + x[j+M]
                y[K+1] = x[j]   - x[j+M]
                y[K+2] = x[j+1] + x[j+1+M]
                y[K+3] = x[j+1] - x[j+1+M]
                K=K+4
            J=J+2*M
        x=y.copy()
        L=L+1
 
    y=x/float(N)
    
    return y

def fastfood_forkernel(X,para,sgm,use_spiral=False):
    d0, m = X.shape
#    print('d0 =',d0,', m =',m)
    l = int(np.ceil(np.log2(d0)))
    d = 2**l
    if d == d0:
        XX = X
    else:
        XX = np.zeros((d,m))
        XX[0:d0,:] = X
#    print('Shape of XX:',XX.shape)
    
    k = len(para.B)
    n = d*k
    tht = np.zeros((n,m))
    for ii in xrange(k):
        B = para.B[ii]
        G = para.G[ii]
        PI = para.PI[ii]
#        XX = np.dot(B,XX)
        XX = (B * XX.T).T
        T = FWHT(XX)
        T = T[PI,:]
#        print('T.shape:',T.shape,',(G*d).shape:',(G*d).shape)
        T = (G*d) * T.T
#        print('T.shape:',T.shape)
        T = FWHT(T)
#        print('T.shape:',T.shape)
        idx1 = ii*d
        idx2=(ii+1)*d
#        print('idx1=',idx1,',idx2=',idx2)
        tht[idx1:idx2,:] = T.T
        
    S = para.S
    tht = (S*np.sqrt(d) * tht.T).T
    
    T = tht/sgm
    phi = np.concatenate([np.cos(T),np.sin(T)],axis=1)
    phi = 1.0/np.sqrt(n) * phi
    return phi,tht

In [85]:
para = fastfood_params(n,d)
#print('Fastfood params:',params)

phi1 = fastfood_forkernel(X1,para,sgm)
print('phi1 =',phi1,', len(phi1) =',len(phi1))
K_appro = np.dot(phi1[0].T,phi1[1])
print('K_appro =',K_appro)

phi2 = fastfood_forkernel(X2,para,sgm)
print('phi2 =',phi2,', len(phi2) =',len(phi2))

phi1 = (array([[ 0.0143086 ,  0.02209628, -0.0168388 , -0.00018827],
       [ 0.01179164,  0.01947134,  0.01868792,  0.01044739],
       [-0.00777528, -0.02191002, -0.02068396,  0.0028692 ],
       ..., 
       [ 0.02207265,  0.02205543,  0.00103895, -0.00135627],
       [-0.0072543 ,  0.00929255, -0.02087238,  0.02004819],
       [ 0.01864449,  0.01743471, -0.01186019,  0.01357616]]), array([[ -8.66453361,  -0.08520092],
       [ 10.07910711,   4.92459932],
       [ 43.52821879,  30.11379979],
       ..., 
       [  0.47034914,  -0.61416484],
       [ 43.77893936,  11.36763913],
       [ -5.66558038,   6.61608621]])) , len(phi1) = 2
K_appro = [[  16.43236492   17.61847467]
 [ -23.91403045  -34.13257537]
 [ 188.79723272    4.38632873]
 [  45.82851488  165.06487688]]


AssertionError: 